In [1]:
#import mariadb
import tqdm
import pickle
import pandas as pd
import mysql.connector
from decimal import Decimal
from neo4j import GraphDatabase
from datetime import timedelta

In [2]:
df_pw = pd.read_csv("pw.csv", delimiter=",")

In [3]:
from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL
import pymysql
db_url = {
    'database': 'DarknetScrape',
    'drivername': 'mysql',
    'username': df_pw["name"][0],
    'password': df_pw["password"][0],
    'host': '127.0.0.1',
    'port': 3306,
    'query': {'charset': 'utf8'},
}

db_connection = create_engine(URL(**db_url), encoding="utf8")

c:\users\d067608\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:14: SADeprecationWarning: Calling URL() directly is deprecated and will be disabled in a future release.  The public constructor for URL is now the URL.create() method.
  


In [4]:
db = GraphDatabase.driver("neo4j://127.0.0.1:7687", auth=(df_pw["name"][0], df_pw["password"][0]))
session = db.session()

In [5]:
j_addresses=pd.read_csv('candidates_final.csv')
j_addresses=j_addresses.drop(columns=['Unnamed: 0'])
j_addresses.head(1)

,Vendor,Address
0,CNSHarmony,34Xo7fKRe9MjYBf3tyJjAFg9qX2kuFuhA1


In [6]:
query = """select a.vendor_id, b.username from (
SELECT vendor_id, count(*) as count
from vendors
group by vendor_id
) as a
JOIN (SELECT vendor_id, username from vendors where market = 'Cryptonia') b
on a.vendor_id = b.vendor_id
where a.count = 2;
"""
df = pd.read_sql(query, con=db_connection)
df=df[df['username'].isin(j_addresses['Vendor'])]
j_addresses = j_addresses[j_addresses["Vendor"].isin(df["username"])]
j_addresses_id = pd.merge(j_addresses,df,left_on='Vendor',right_on='username')
j_addresses_id=j_addresses_id[["Vendor","Address","vendor_id"]]
j_addresses_id.head(3)

,Vendor,Address,vendor_id
0,GreenSupreme,bc1qcuxc8fl7eqm8a48sv3n6vgqhc0mtzsswdy4h66,135137a5e88e1fa749c1374f51048773
1,SocialPharma,3DdUb8Tt59tvZacikVt6R9pYWYqzGqgxff,1db74355bd33521d281b3e15cc6dbe21
2,americansteroids,1PWVgJ96YtELbfqEW625JPG16NXsFYU315,26bda6b380be1d4321145f15599f2508


In [7]:
def get_cospending_addresses(address, session):
    neo_query = session.run("""
            MATCH (vendor:Address)
                WHERE vendor.address = '""" + address + """'
            WITH vendor
            MATCH (vendor)-[r:SENDS]->(t:Transaction)
            WITH t
            MATCH (vendor2:Address)-[s:SENDS]->(t)

            RETURN vendor2.address
            """)
    wallet = set()
    for r in neo_query:
        wallet.add(r[0])
        wallet.add(get_cospending_addresses(r[0],session))
    return wallet


In [8]:
# vendor_wallets = pd.DataFrame()
# for i in j_addresses_id.head(2).itertuples():
#     big_wallet = set()
#     wallet = get_cospending_addresses(i.Address,session)


In [12]:
vendor_wallets = pd.DataFrame()
for i in j_addresses_id.head(2).itertuples():
    wallet = pd.DataFrame(data = [[i.Vendor,i.Address,i.vendor_id]], columns=["Vendor","Address","vendor_id"])
    reverse_wallet = wallet

    while len(reverse_wallet) > 0:
        address = reverse_wallet["Address"][0]
        print(address)
        neo_query = session.run("""
                MATCH (vendor:Address)
                    WHERE vendor.address = '""" + address + """'
                WITH vendor
                MATCH (vendor)-[r:SENDS]->(t:Transaction)
                WITH t
                MATCH (vendor2:Address)-[s:SENDS]->(t)

                RETURN vendor2.address
                """)
        wallet_temp = set()
        for r in neo_query:
            wallet_temp.add(r[0])
        
        if len(wallet_temp) > 0:
            df_neo4j = pd.DataFrame(wallet_temp)
            df_neo4j["Vendor"] = i.Vendor
            df_neo4j["Address"] = df_neo4j[0]
            df_neo4j["vendor_id"] = i.vendor_id
            df_neo4j = df_neo4j[["Vendor","Address","vendor_id"]]

            wallet = pd.concat([wallet,df_neo4j[~df_neo4j["Address"].isin(wallet["Address"])]]).reset_index(drop=True)
            reverse_wallet = pd.concat([reverse_wallet,df_neo4j[~df_neo4j["Address"].isin(wallet["Address"])]]).reset_index(drop=True)
        reverse_wallet = reverse_wallet.iloc[1:]
        

    
    vendor_wallets = pd.concat([vendor_wallets,wallet]).reset_index(drop=True)
    
display(vendor_wallets.head())

bc1qcuxc8fl7eqm8a48sv3n6vgqhc0mtzsswdy4h66


StopIteration: 

In [11]:
reverse_wallet.iloc[1:]

,Vendor,Address,vendor_id


In [ ]:
len(vendor_wallets)

In [ ]:
vendor_wallets.to_csv("vendor_wallets.csv")